# Imports :

In [ ]:
# Skipping version problems :
! pip install --upgrade numpy
# Fixing non-readable DCIM images issue :

! pip install python-gdcm
! pip install pylibjpeg pylibjpeg-libjpeg pydicom

In [ ]:
import os
import glob
import gc

import numpy as np
import pandas as pd
from scipy import ndimage
import tensorflow as tf

from joblib import Parallel, delayed
from tqdm.notebook import tqdm

import pydicom as dicom
import gdcm


import nibabel as nib

# Defining directories :

In [ ]:
# Checking current dir :

INITIAL_PATH = os.getcwd()
print(INITIAL_PATH)

In [ ]:
## Defining each useful path :

# Where the images are :
TRAIN_IMAGES_PATH = '/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images/'
TEST_IMAGES_PATH = '/kaggle/input/rsna-2022-cervical-spine-fracture-detection/test_images/'

# Pathes of the dataframes :
TRAIN_CSV_PATH = '/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train.csv'
TEST_CSV_PATH = '/kaggle/input/rsna-2022-cervical-spine-fracture-detection/test.csv'

# Where we are going to save the preprocessed data :
TRAIN_OUTPUT_PATH = './train_arrays/'
TEST_OUTPUT_PATH = './test_arrays/'
if not os.path.exists(TRAIN_OUTPUT_PATH): os.mkdir(TRAIN_OUTPUT_PATH)
if not os.path.exists(TEST_OUTPUT_PATH): os.mkdir(TEST_OUTPUT_PATH)  

# Defining other useful features :


In [ ]:
# Defining desired width and depth of the output arrays :

desired_width = 64
desired_height = 64
desired_depth = 64

In [ ]:
# Putting images paths into lists : 

train_images = os.listdir(TRAIN_IMAGES_PATH)
test_images = os.listdir(TEST_IMAGES_PATH)

In [ ]:
# Reading dataframes :

train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)

In [ ]:
# List of IDs of every patients :

train_patients = train_df.StudyInstanceUID.to_list()

# Creating the preparation steps :

In [ ]:
### Creating a function to load dicom files, even if compressed :

def load_dicom(path: str):
    """Load a dicom file (.dcm) even if it is compressed."""
    try:
        file_as_array = dicom.dcmread(path).pixel_array
    except:
        decompressed_file = gdcm.ImageReader().SetFileName(path).Read()
        file_as_array = decompressed_file.pixel_array
    return(file_as_array)

In [ ]:
# Creating the preprocessing function including loading dicom normalization and resize :

def preprocessing_slice(slice_path: str):
    """Load dicom of a slice, normalize and resize it"""
    # Loading dicom :
    slice_array = load_dicom(slice_path)
    slice_array = slice_array.astype(np.uint8)
    
    # Normalization :
    slice_array = slice_array - np.min(slice_array)
    if np.max(slice_array) != 0:
        slice_array = slice_array / np.max(slice_array)
    slice_array = (slice_array * 255).astype(np.uint8)
        
    # Resize (2D) :  
    width_factor = desired_width / slice_array.shape[0]
    height_factor = desired_height / slice_array.shape[1]
    
    slice_array = ndimage.zoom(slice_array, (width_factor, height_factor), order=3) # resize with spline interpolation of order 3
    
    return(slice_array)

In [ ]:
def resize_depth(numpy_volume: np.array, desired_depth=desired_depth):
    """Resize across z-axis"""
    ## Get current depth
    current_depth = numpy_volume.shape[0]
    ## Compute depth factor
    depth_factor = desired_depth / current_depth
    
    ## Resize across z-axis
    # Rotate
    numpy_volume = ndimage.rotate(numpy_volume, 90, reshape=False)
    # Resize
    volume = ndimage.zoom(numpy_volume, (depth_factor, 1, 1), order=1) # resize with spline interpolation of order 1
    return volume

In [ ]:
# Creating a function to parallelize most of the charge of loading the dicom files :

def load_and_stack_dicom_parallel(scan_path: str):
    """Load all dicom files from a scan and stack them all in a numpy array"""
    # Defining slice paths :
    slice_paths = sorted(glob.glob(os.path.join(scan_path, "*")),
                         key=lambda x: int(x.split('/')[-1].split(".")[0]))
    
    # Preprocessing slices :
    images = Parallel(n_jobs=-1)(delayed(preprocessing_slice)(filename) for filename in slice_paths)
    
    # Returning stacked slices as a resized on depth (3rd dimension) volume :
    return(tf.expand_dims(resize_depth(np.array(images)), axis=3))

# Creating preprocessed data :

In [ ]:
# Function to create and save the 3D volumes corresponding to a scan :

def save_3D_arrays(scan_path: str, output_path: str):
    """Create and save the 3D arrays corresponding to a scan"""
    
    # Preprocessing and creation :
    volume = load_and_stack_dicom_parallel(scan_path=scan_path)
    
    # Saving the numpy array :
    volume_file_name = output_path + scan_path.split('/')[-1] + '.npy'
    np.save(volume_file_name, volume)
    
    # Deleting in memory :
    del volume
    
    return None

In [ ]:
# Creation of the preprocessed array volumes :

for i in tqdm(range(len(train_patients))):
    case_path = TRAIN_IMAGES_PATH + train_patients[i]
    save_3D_arrays(case_path, TRAIN_OUTPUT_PATH)


# Free up memory :

gc.collect()